pip install transformers[torch] datasets evaluate opendatasets

In [7]:
import os
import mlflow
import datasets
import transformers
import evaluate
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, TrainingArguments, Trainer, EvalPrediction
import opendatasets as od
import pandas as pd
import numpy as np
import pickle as pkl
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import torch

In [2]:
od.download(
    "https://www.kaggle.com/datasets/rmisra/politifact-fact-check-dataset/data"
    )
data = datasets.Dataset.from_json("politifact-fact-check-dataset/politifact_factcheck_data.json")
data = data.train_test_split(train_size=0.8)

Using custom data configuration default-58a22729889bbb0b
Found cached dataset json (C:/Users/Abelda-san/.cache/huggingface/datasets/json/default-58a22729889bbb0b/0.0.0)


Skipping, found downloaded files in ".\politifact-fact-check-dataset" (use force=True to force download)


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

label2id = {
    'true': 0,
    'mostly-true': 1,
    'half-true': 2,
    'mostly-false': 3,
    'false': 4,
    'pants-fire': 5
}
id2label = {
    0: 'true',
    1: 'mostly-true',
    2: 'half-true',
    3: 'mostly-false',
    4: 'false',
    5: 'pants-fire'
}
num_labels = len(label2id)

def preprocess_data(examples):

    text = examples["statement"]
    labels = pd.Series(examples['verdict'])

    encoding = tokenizer(text, padding=True, truncation=True)
    encoding['labels'] = list(labels.apply(lambda x: [1.0 if label2id[x]==i else 0.0 for i in range(num_labels)]))

    return encoding


train_tokenized_dataset = data["train"].map(preprocess_data, batched=True, remove_columns=data["train"].column_names)
test_tokenized_dataset = data["test"].map(preprocess_data, batched=True, remove_columns=data["test"].column_names)

train_tokenized_dataset.set_format("torch")
test_tokenized_dataset.set_format("torch")

 80%|████████  | 4/5 [00:00<00:00,  8.36ba/s]


In [6]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                            problem_type="multi_label_classification", 
                                                            num_labels=6,
                                                            id2label=id2label,
                                                            label2id=label2id)

Downloading: 100%|██████████| 268M/268M [02:48<00:00, 1.59MB/s] 
c:\Users\Abelda-san\anaconda3\envs\factcheck\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Abelda-san\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at distilbert-base-unc

In [8]:
training_args = TrainingArguments(
    f"distilbert_finetuning-factchecking",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=15,
    per_device_eval_batch_size=15,
    num_train_epochs=5,
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_dataset,
    eval_dataset=test_tokenized_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [9]:
trainer.train()

c:\Users\Abelda-san\anaconda3\envs\factcheck\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16921
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 5645
  1%|          | 47/5645 [04:15<8:24:28,  5.41s/it]